# Assistants function calling with Bing Search
In this notebook, we'll show how you can use the [Bing Search APIs](https://www.microsoft.com/bing/apis/llm) and [function calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=python) to ground Azure OpenAI models on data from the web. This is a great way to give the model access to up to date data from the web.

You'll need to create a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) before you begin.

### Time

You should expect to spend 10 minutes running this sample.

### Before you begin
#### Installation
The following packages are required to execute this notebook.

### Parameters
Update the following config to include details of your Azure OpenAI and Bing Search resources

In [47]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")

Found Azure OpenAI API Base Endpoint: https://swed-oai.openai.azure.com/


In [48]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2024-02-15-preview"
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_MODEL")
bing_search_subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"

### Get guest name from file

In [49]:
# Open the file
with open('guest_name.txt', 'r') as f:
    guest = f.read()

# Print the content
print(guest)

Neil deGrasse Tyson


### Define a function to call the Bing Search APIs
To learn more about using the Bing Search APIs with Azure OpenAI, see [Bing Search APIs, with your LLM](https://learn.microsoft.com/bing/search-apis/bing-web-search/use-display-requirements-llm).

In [50]:
import json
import requests
import time

from openai import AzureOpenAI
from pathlib import Path
from typing import Optional


def search(query: str) -> list:
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_search_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)

### Test Bing Search

In [51]:
search("Who is "+guest + "?")

'[{"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (US: / d \\u0259 \\u02c8 \\u0261 r \\u00e6 s / d\\u0259-GRASS or UK: / d \\u0259 \\u02c8 \\u0261 r \\u0251\\u02d0 s / d\\u0259-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator. Tyson studied at Harvard University , the University of Texas at Austin , and Columbia University ."}, {"title": "Neil deGrasse Tyson | Biography, Books, TV Shows, & Facts", "link": "https://www.britannica.com/biography/Neil-deGrasse-Tyson", "snippet": "Neil deGrasse Tyson (born October 5, 1958, New York, New York, U.S.) American astronomer who popularized science with his books and frequent appearances on radio and television. (Read Neil deGrasse Tyson\\u2019s Britannica essay on public science.)"}, {"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://nl.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse 

### Get things running end to end
In the following cells, we will define some functions essential for assistants with function calling. All these functions come together in our final cells, where we will define a new web search assistant, give it instructions on its functionality and ask a question.

In [52]:
def poll_run_till_completion(
    client: AzureOpenAI,
    thread_id: str,
    run_id: str,
    available_functions: dict,
    verbose: bool,
    max_steps: int = 10,
    wait: int = 3,
) -> None:
    """
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param run_id: Run ID
    @param assistant_id: Assistant ID
    @param verbose: Print verbose output
    @param max_steps: Maximum number of steps to poll
    @param wait: Wait time in seconds between polls

    """

    if (client is None and thread_id is None) or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if verbose:
                print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            if run.status == "requires_action":
                tool_responses = []
                if (
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response})

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

    except Exception as e:
        print(e)

In [53]:
def create_message(
    client: AzureOpenAI,
    thread_id: str,
    role: str = "",
    content: str = "",
    file_ids: Optional[list] = None,
    metadata: Optional[dict] = None,
    message_id: Optional[str] = None,
) -> any:
    """
    Create a message in a thread using the client.

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param role: Message role (user or assistant)
    @param content: Message content
    @param file_ids: Message file IDs
    @param metadata: Message metadata
    @param message_id: Message ID
    @return: Message object

    """
    if metadata is None:
        metadata = {}
    if file_ids is None:
        file_ids = []

    if client is None:
        print("Client parameter is required.")
        return None

    if thread_id is None:
        print("Thread ID is required.")
        return None

    try:
        if message_id is not None:
            return client.beta.threads.messages.retrieve(thread_id=thread_id, message_id=message_id)

        if file_ids is not None and len(file_ids) > 0 and metadata is not None and len(metadata) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, file_ids=file_ids, metadata=metadata
            )

        if file_ids is not None and len(file_ids) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, file_ids=file_ids
            )

        if metadata is not None and len(metadata) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, metadata=metadata
            )

        return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content)

    except Exception as e:
        print(e)
        return None

In [54]:
def retrieve_and_print_messages(
    client: AzureOpenAI, thread_id: str, verbose: bool, out_dir: Optional[str] = None
) -> any:
    """
    Retrieve a list of messages in a thread and print it out with the query and response

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param verbose: Print verbose output
    @param out_dir: Output directory to save images
    @return: Messages object

    """

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {"user": "User query", "assistant": "Assistant response"}

        prev_role = None

        if verbose:
            print("\n\nCONVERSATION:")
        for md in reversed(messages.data):
            if prev_role == "assistant" and md.role == "user" and verbose:
                print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == "text":
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == "image_file":
                    image_data = client.files.content(mc.image_file.file_id)
                    if out_dir is not None:
                        out_dir_path = Path(out_dir)
                        if out_dir_path.exists():
                            image_path = out_dir_path / (mc.image_file.file_id + ".png")
                            with image_path.open("wb") as f:
                                f.write(image_data.read())

                if verbose:
                    if prev_role == md.role:
                        print(txt_val)
                    else:
                        print("{}:\n{}".format(display_role[md.role], txt_val))
            prev_role = md.role
        return messages
    except Exception as e:
        print(e)
        return None

### Define parameters and tools for the Assistant

In [55]:
name = "websearch-assistant"
instructions = """You are an assistant designed to help people answer questions.

You have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]


### Create user request / prompt

In [56]:
message = {"role": "user", "content": "Who is "+guest + "?"}

### Call the Assistant

In [57]:
available_functions = {"search_bing": search}
verbose_output = True

client = AzureOpenAI(api_key=aoai_api_key, api_version=api_version, azure_endpoint=azure_endpoint)

assistant = client.beta.assistants.create(
    name=name, description="", instructions=instructions, tools=tools, model=deployment_name
)

thread = client.beta.threads.create()
create_message(client, thread.id, message["role"], message["content"])


run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id, instructions=instructions)
poll_run_till_completion(
    client=client, thread_id=thread.id, run_id=run.id, available_functions=available_functions, verbose=verbose_output
)
messages = retrieve_and_print_messages(client=client, thread_id=thread.id, verbose=verbose_output)

Poll 0: queued
Poll 1: in_progress
Poll 2: in_progress
Poll 3: in_progress
Poll 4: in_progress
Poll 5: in_progress
Poll 6: in_progress
Poll 7: completed


CONVERSATION:
User query:
Who is Neil deGrasse Tyson?
Assistant response:
Neil deGrasse Tyson is an American astrophysicist, author, and science communicator. He was born on October 5, 1958, in Manhattan, New York, NY. He is well known for his role as the host of the television show "Cosmos: A Spacetime Odyssey," which is a reboot of Carl Sagan's original series "Cosmos: A Personal Voyage."

Tyson's research has focused on star formation, dwarf galaxies, and the Milky Way's structure. He has written several books about astronomy and science for the general public, making complex concepts accessible to non-scientists.

Tyson is also the Frederick P. Rose Director of the Hayden Planetarium at the Rose Center for Earth and Space in New York City. He has been a regular guest on talk shows, radio shows, and other media outlets, discussing

### Extract text value out of the response

In [58]:
bio = messages.data[0].content[0].text.value
print(bio)

Neil deGrasse Tyson is an American astrophysicist, author, and science communicator. He was born on October 5, 1958, in Manhattan, New York, NY. He is well known for his role as the host of the television show "Cosmos: A Spacetime Odyssey," which is a reboot of Carl Sagan's original series "Cosmos: A Personal Voyage."

Tyson's research has focused on star formation, dwarf galaxies, and the Milky Way's structure. He has written several books about astronomy and science for the general public, making complex concepts accessible to non-scientists.

Tyson is also the Frederick P. Rose Director of the Hayden Planetarium at the Rose Center for Earth and Space in New York City. He has been a regular guest on talk shows, radio shows, and other media outlets, discussing the significance of space and science in the modern day.


### Save the biography in file

In [59]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)